In [1]:
#import libraries

import scipy.sparse as sps
import numpy as np
import os
import pandas as pd 

URM_all= sps.load_npz("./Processed Matrices/URM_simple_coo_2020.npz").tocsr()
ICM_all= sps.load_npz("./Processed Matrices/ICM_simple_coo_2020.npz").tocsr()

In [2]:
%load_ext Cython

In [3]:
from Notebooks_utils.evaluation_function import evaluate_algorithm
from Base.Similarity.Compute_Similarity_Python import Compute_Similarity_Python
from Base.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

np.random.seed(41148)
URM_train, URM_valid = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.95)

In [4]:
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython

In [5]:
#evaluator_validation = EvaluatorHoldout(URM_valid, cutoff_list=[10])
recommender= SLIM_BPR_Cython(URM_all)

SLIM_BPR_Recommender: URM Detected 1079 (4.15 %) cold items.


In [6]:
recommender.fit(batch_size = 1000, lambda_i = 0.00044253503429704934, lambda_j = 0.0005578499833209431, learning_rate = 0.0012090203681385636, topK = 703,
            sgd_mode="adam", gamma=0.995,epochs=155)

SLIM_BPR_Recommender: Automatic selection of fastest train mode. Available RAM is 12292.00 MB (77.61%) of 15839.00 MB, required is 2698.80 MB. Using dense matrix.
Processed 7947 ( 100.00% ) in 0.52 seconds. BPR loss is 1.50E-04. Sample per second: 15250
SLIM_BPR_Recommender: Epoch 1 of 155. Elapsed time 0.02 sec
Processed 7947 ( 100.00% ) in 0.54 seconds. BPR loss is 1.41E-03. Sample per second: 14680
SLIM_BPR_Recommender: Epoch 2 of 155. Elapsed time 0.04 sec
Processed 7947 ( 100.00% ) in 0.57 seconds. BPR loss is 3.36E-03. Sample per second: 13893
SLIM_BPR_Recommender: Epoch 3 of 155. Elapsed time 0.07 sec
Processed 7947 ( 100.00% ) in 0.60 seconds. BPR loss is 4.87E-03. Sample per second: 13305
SLIM_BPR_Recommender: Epoch 4 of 155. Elapsed time 0.09 sec
Processed 7947 ( 100.00% ) in 0.62 seconds. BPR loss is 7.59E-03. Sample per second: 12778
SLIM_BPR_Recommender: Epoch 5 of 155. Elapsed time 0.12 sec
Processed 7947 ( 100.00% ) in 0.65 seconds. BPR loss is 1.18E-02. Sample per secon

In [7]:
rec_df=pd.DataFrame({"user_id":[],"item_list":[]})
target_user_list=list(pd.read_csv("./Data/2020/data_target_users_test.csv").user_id)

for uid in target_user_list:
    rec_list=map(str,recommender.recommend(uid)[:10])
    rec_df=rec_df.append({"user_id":uid,"item_list":' '.join(rec_list)},ignore_index=True)

    
rec_df=rec_df.astype({"user_id":'int32',"item_list":'str'})
rec_df=rec_df.set_index('user_id')
rec_df.to_csv("SLIM_BPR_recommendations.csv")